# Installation and Setup

In [1]:
# !pip install accelerate

# Create .env file at cookbooks/Function-Calling-OpenAI/python/.env containing the 
# following line: OPENAI_API_KEY=<your key here>
# You can get your key from https://platform.openai.com/api-keys 
import openai
import dotenv
import os
dotenv.load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

## Load AI Config `travel.aiconfig.json`
Download file [from here](https://github.com/lastmile-ai/aiconfig/blob/main/cookbooks/Getting-Started/travel.aiconfig.json). Upload to 'Files' in your colab notebook and load (shown below).


In [2]:
from aiconfig import AIConfigRuntime, InferenceOptions, CallbackManager
# import aiconfig_extension_hugging_face_transformers
# from aiconfig_extension_hugging_face_transformers.text_generation import (
#     HuggingFaceTextGenerationTransformer
# )

# Load the aiconfig.
config = AIConfigRuntime.load('travel.aiconfig.json')
config.callback_manager = CallbackManager([])

/opt/homebrew/lib/python3.11/site-packages/pydantic/_internal/_fields.py:128: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Run `get_activities` prompt with streaming




In [3]:
# Run a single prompt (with streaming). inference_options = InferenceOptions(stream=True)

def print_stream(data, _accumulated_data, _index):
    print(data, end="", flush=True)
    print(" --> hey I should be printing something ")

inference_options = InferenceOptions(stream=False, stream_callback=print_stream)
prompts = [
    # "gen_image", 
    "gen_image_sdxl"
    ]
for name in prompts:
    await config.run(name, options=inference_options)
# config.save('new.aiconfig.json')



Building the pipeline... This can take a long time, especially if you haven't created one
before on this machine. Please be patient!

If this seems to be taking too long, you can change some of the pipeline generation
params. See more details here:
https://huggingface.co/docs/diffusers/using-diffusers/loading



Keyword arguments {'safety_checker': None} are not expected by StableDiffusionXLPipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00,  8.37it/s]




Calling image generation. This can take a long time, (up to SEVERAL MINUTES depending
on the model), please hold on...

If this seems to be taking too long, you can change some of the params. For example, 
you can set the `num_inference_steps` to a lower value at the cost of reduced image
quality. See full list of params here:
https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline.__call__

If that doesn't work, you can also try less computationally intensive models. 
        


100%|██████████| 50/50 [21:14<00:00, 25.49s/it]


StableDiffusionXLPipelineOutput(images=[<PIL.Image.Image image mode=RGB size=1024x1024 at 0x2A9472410>, <PIL.Image.Image image mode=RGB size=1024x1024 at 0x2A965D4D0>])
count=0
image=<PIL.Image.Image image mode=RGB size=1024x1024 at 0x2A9472410>
has_nsfw=None
64-encoded-image: 
 ...
count=1
image=<PIL.Image.Image image mode=RGB size=1024x1024 at 0x2A965D4D0>
has_nsfw=None
64-encoded-image: 
 ...


# Run `gen_itinerary` prompt


In [4]:
config.save('new.aiconfig.json')

# Add prompt to config

In [39]:
# Add gen_packing_list as a prompt to config
from aiconfig import Prompt
model = "gpt-3.5-turbo"
data = {
    "model": model,
    "messages": [
        {
            "role": "system",
            "content": "You provide a bulleted list of items to pack for a week long trip."
        },
        {
            "role": "user",
            "content": "What should I bring to {{location}}?"
        }
    ]
}

new_prompts = await config.serialize(model, data, prompt_name="gen_packing_list", params={"location": "nyc"})
config.add_prompt(prompt_name="gen_packing_list", prompt_data=new_prompts[0])

INFO:my-logger:Callback called. event
: name='on_serialize_start' file='aiconfig.Config' data={'gpt-3.5-turbo': 'gpt-3.5-turbo', 'data': {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You provide a bulleted list of items to pack for a week long trip.'}, {'role': 'user', 'content': 'What should I bring to {{location}}?'}]}, 'prompt_name': 'gen_packing_list', 'params': {'location': 'nyc'}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_serialize_start' file='aiconfig.default_parsers.openai' data={'prompt_name': 'gen_packing_list', 'data': {'model': 'gpt-3.5-turbo', 'messages': [{'role': 'system', 'content': 'You provide a bulleted list of items to pack for a week long trip.'}, {'role': 'user', 'content': 'What should I bring to {{location}}?'}]}, 'parameters': {'location': 'nyc'}, 'kwargs': {}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_serialize_complete' file='aiconfig.default_parsers.openai' data={'res

In [40]:
packing_list = await config.run("gen_packing_list", params=None, options=inference_options)

INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.Config' data={'prompt_name': 'gen_packing_list', 'params': None, 'options': <aiconfig.model_parser.InferenceOptions object at 0x78287ee520e0>, 'kwargs': {}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_run_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='gen_packing_list', input='What should I bring to {{location}}?', metadata=PromptMetadata(model=ModelMetadata(name='gpt-3.5-turbo', settings={'system_prompt': {'role': 'system', 'content': 'You provide a bulleted list of items to pack for a week long trip.'}}), tags=None, parameters={'location': 'nyc'}, remember_chat_context=True), outputs=[]), 'options': <aiconfig.model_parser.InferenceOptions object at 0x78287ee520e0>, 'parameters': {}} ts_ns=1701177332281242097
INFO:my-logger:Callback called. event
: name='on_deserialize_start' file='aiconfig.default_parsers.openai' data={'prompt': Prompt(name='gen_packin

Here's a list of items you should consider packing for your trip to NYC:

1. Comfortable walking shoes: NYC is a city best explored on foot, so be sure to pack comfortable shoes for all the walking you'll be doing.
2. Lightweight clothing: Depending on the season, pack layered clothing that can accommodate fluctuating temperatures. Summers can be hot and humid while winters can be cold and snowy.
3. Weather-appropriate outerwear: Bring a waterproof jacket or umbrella, especially if you're visiting during the rainy season. In colder months, pack a warm coat, hat, and gloves.
4. Daypack or tote bag: Carry a small bag to hold your personal items, snacks, water bottle, and any souvenirs you might pick up during the day.
5. Portable phone charger: With all the sightseeing and navigating you'll be doing, it's important to ensure your phone stays charged throughout the day.
6. Travel guide or map: Even with smartphones, having a physical map or guidebook can be handy for quick reference or if

INFO:my-logger:Callback called. event
: name='on_run_complete' file='aiconfig.default_parsers.openai' data={'result': [ExecuteResult(output_type='execute_result', execution_count=0, data={'content': "Here's a list of items you should consider packing for your trip to NYC:\n\n1. Comfortable walking shoes: NYC is a city best explored on foot, so be sure to pack comfortable shoes for all the walking you'll be doing.\n2. Lightweight clothing: Depending on the season, pack layered clothing that can accommodate fluctuating temperatures. Summers can be hot and humid while winters can be cold and snowy.\n3. Weather-appropriate outerwear: Bring a waterproof jacket or umbrella, especially if you're visiting during the rainy season. In colder months, pack a warm coat, hat, and gloves.\n4. Daypack or tote bag: Carry a small bag to hold your personal items, snacks, water bottle, and any souvenirs you might pick up during the day.\n5. Portable phone charger: With all the sightseeing and navigating y

# 5. Save the AIConfig with outputs


In [37]:
# Save the aiconfig to disk. and serialize outputs from the model run
config.save('updated.aiconfig.json', include_outputs=True)

# 6. Open the AIConfig in AI Workbook Playground
1. Download `updated.aiconfig.json`.
2. Go to https://lastmileai.dev.
3. Go to Workbooks page: https://lastmileai.dev/workbooks.
4. Click dropdown '+New Workbook' and select 'Create from AIConfig'
5. Upload `updated.aiconfig.json`.

Try out the workbook playground here: [NYC Travel Workbook](https://lastmileai.dev/workbooks/clooqs3p200kkpe53u6n2rhr9)